In [45]:
# !pip install pytorch-lightning

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from collections import Counter
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

import pytorch_lightning as pl

from tqdm import tqdm_notebook as tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
train = pd.read_csv('../dataset/경진대회/train_data_dataframe.csv')
test = pd.read_csv('../dataset/경진대회/test_data_dataframe.csv')

In [5]:
train['text'] =  train['context'] + train['title'] + train['question']
test['text'] = test['context'] + test['title']  + test['question']

In [49]:
train.head(2)

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재..."
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...


In [6]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
train['uncased_text'] = train['text'].apply(lambda x: x.lower())
test['uncased_text'] = test['text'].apply(lambda x: x.lower())


In [8]:
train.isnull().sum()

content_id          0
title               0
paragraph_id        0
context             0
question_id         0
question            0
answer_text      9305
answer_start     9305
is_impossible       0
text                0
uncased_text        0
dtype: int64

In [9]:
# Tokenize
train['tokenized_text'] = train['uncased_text'].apply(tokenizer.tokenize)
test['tokenized_text'] = test['uncased_text'].apply(tokenizer.tokenize)

train = train.dropna()
train['tokenized_selected_text'] = train['answer_text'].apply(tokenizer.tokenize)

In [10]:
train.head()

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text,uncased_text,tokenized_text,tokenized_selected_text
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","[ᄋ, ##ᅵ, ᄀ, ##ᅳ, ##ᆯ, ##ᄋ, ##ᅦ, ##ᄉ, ##ᅥ, ##ᄂ,...","[ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄌ, ##..."
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,현위치와 관련된 평가부문은 imd 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,"[[UNK], [UNK], ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄇ, ##ᅮ...","[ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅲ, ##ᆫ, [UNK]]"
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,imd의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,"[im, ##d, ##ᄋ, ##ᅴ, ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄂ...","[ᄌ, ##ᅩ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆸ, ##ᄀ, ##ᅪ, ##ᄒ, ##..."
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","[ᄀ, ##ᅵ, ##ᄋ, ##ᅥ, ##ᆸ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄋ, ##...","[ᄋ, ##ᅧ, ##ᆫ, ##ᄀ, ##ᅩ, ##ᆼ, ##ᄉ, ##ᅥ, ##ᄋ, ##..."
5,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_Ldm3nIToPd,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...",QUES_OhH70koIzZ,과학기술의 성과로 나타나는 기술집약제품이 수출에서 차지하는 비중 등을 바탕으로 하여...,산출지표,348.0,False,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","imd의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","[im, ##d, ##ᄋ, ##ᅴ, ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ...","[ᄉ, ##ᅡ, ##ᆫ, ##ᄎ, ##ᅮ, ##ᆯ, ##ᄌ, ##ᅵ, ##ᄑ, ##ᅭ]"


In [11]:
# Filter train data
start_position_candidates = []
end_position_candidates = []
train['select_length'] = train['tokenized_selected_text'].map(len)

for i in tqdm(range(len(train))):
    start_position_candidate = [j for j, tok in enumerate(train['tokenized_text'].iloc[i]) if tok == train['tokenized_selected_text'].iloc[i][0]]
    end_position_candidate = [j for j, tok in enumerate(train['tokenized_text'].iloc[i]) if tok == train['tokenized_selected_text'].iloc[i][-1]]

    start_position_candidate = [idx for idx in start_position_candidate if idx + train['select_length'].iloc[i] - 1 in end_position_candidate]
    end_position_candidate = [idx for idx in end_position_candidate if idx - train['select_length'].iloc[i] + 1 in start_position_candidate]

    start_position_candidates.append(start_position_candidate)
    end_position_candidates.append(end_position_candidate)

/tmp/ipykernel_3883803/140825938.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(len(train))):


  0%|          | 0/28024 [00:00<?, ?it/s]

In [12]:
start_position_candidates = [l[0] if len(l) > 0 else -1 for l in start_position_candidates]
end_position_candidates = [l[0] if len(l) > 0 else -1 for l in end_position_candidates]

In [13]:
train['start_position'] = start_position_candidates
train['end_position'] = end_position_candidates
test['start_position'] = -1
test['end_position'] = -1

In [14]:
train

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text,uncased_text,tokenized_text,tokenized_selected_text,select_length,start_position,end_position
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","[ᄋ, ##ᅵ, ᄀ, ##ᅳ, ##ᆯ, ##ᄋ, ##ᅦ, ##ᄉ, ##ᅥ, ##ᄂ,...","[ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄌ, ##...",14,20,33
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,현위치와 관련된 평가부문은 imd 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,"[[UNK], [UNK], ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄇ, ##ᅮ...","[ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅲ, ##ᆫ, [UNK]]",7,653,659
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,imd의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,"[im, ##d, ##ᄋ, ##ᅴ, ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄂ...","[ᄌ, ##ᅩ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆸ, ##ᄀ, ##ᅪ, ##ᄒ, ##...",24,11,34
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","[ᄀ, ##ᅵ, ##ᄋ, ##ᅥ, ##ᆸ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄋ, ##...","[ᄋ, ##ᅧ, ##ᆫ, ##ᄀ, ##ᅩ, ##ᆼ, ##ᄉ, ##ᅥ, ##ᄋ, ##...",13,173,185
5,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_Ldm3nIToPd,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...",QUES_OhH70koIzZ,과학기술의 성과로 나타나는 기술집약제품이 수출에서 차지하는 비중 등을 바탕으로 하여...,산출지표,348.0,False,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","imd의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","[im, ##d, ##ᄋ, ##ᅴ, ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ...","[ᄉ, ##ᅡ, ##ᆫ, ##ᄎ, ##ᅮ, ##ᆯ, ##ᄌ, ##ᅵ, ##ᄑ, ##ᅭ]",10,160,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37320,CNTS_2129305745,국역 금대전책,PARS_cFgFiF9icI,"실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...",QUES_CvNv5UjUIR,우주 가장 아래에서 금성과 수성 외에 항성까지 서로 가릴 때 모든 것을 먹는 것은 뭐지,달,361.0,False,"실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...","실제 측량해 본 것으로 말씀드리면, 종동(宗動)이 가장 높고, 항성(恒星) 토성(土...","[ᄉ, ##ᅵ, ##ᆯ, ##ᄌ, ##ᅦ, [UNK], ᄇ, ##ᅩ, ##ᆫ, [U...","[ᄃ, ##ᅡ, ##ᆯ]",3,161,163
37321,CNTS_2129305745,국역 금대전책,PARS_R9RLmyTzUy,"신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...",QUES_MrMnyucG4Y,사람이 땅바닥에 있으면서 북쪽으로 이동을 얼마나 해야 북극의 고도가 1도 높아질까,250리,184.0,False,"신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...","신이 생각하기에, 땅은 계란의 노른자와 같고, 하늘은 계란의 껍질과 같으며, 경도(...","[ᄉ, ##ᅵ, ##ᆫ, ##ᄋ, ##ᅵ, ᄉ, ##ᅢ, ##ᆼ, ##ᄀ, ##ᅡ,...","[250, ##ᄅ, ##ᅵ]",3,214,216
37324,CNTS_2129305745,국역 금대전책,PARS_ma0DB2fJSk,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,QUES_XM4RUoboD7,역상을 알지 못함을 분개하면서 이에 대한 견해를 토로하도록 하는 책문을 정조가 내놓...,천체론,1.0,False,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,｢천체론｣은 정조가 하늘의 작용이 민사(民事)와 관계되므로 국가의 다스림에서 천문에...,"[[UNK], ᄎ, ##ᅥ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ, ...","[ᄎ, ##ᅥ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ]",8,1,8
37325,CNTS_2129305745,국역 금대전책,PARS_wGlvzy6QkR,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,QUES_YsPrUyg1xE,사대부들이 정학을 기피하면서 기교에 기반한 글쓰기를 하자 문체반정을 추진한 왕은 누구야,정조,244.0,False,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,｢문체론｣은 국왕 정조가 “문장은 한 시대의 체제가 있어서 세상의 도와 함께 부침한...,"[[UNK], ᄆ, ##ᅮ, ##ᆫ, ##ᄎ, ##ᅦ, ##ᄅ, ##ᅩ, ##ᆫ, ...","[ᄌ, ##ᅥ, ##ᆼ, ##ᄌ, ##ᅩ]",5,19,23


In [15]:
train.to_csv('../dataset/경진대회/process_train.csv')
test.to_csv('../dataset/경진대회/process_test.csv')

In [16]:
df_train, df_val = train_test_split(train, train_size=0.8)

In [18]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [19]:
train.head()

,content_id,title,paragraph_id,context,question_id,question,answer_text,answer_start,is_impossible,text,uncased_text,tokenized_text,tokenized_selected_text,select_length,start_position,end_position
0,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_RrlDreD2hE,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...",QUES_CHGHl0CHHK,경쟁 상대국을 연구하고 경쟁 상대국을 앞설 수 있는 방법을 연구하기 위해서 더욱 활...,기술경쟁력,603.0,False,"이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","이 글에서는 제안한 기술경쟁력 평가 방법의 특징은 두 가지로 요약된다. 첫째, 현재...","[ᄋ, ##ᅵ, ᄀ, ##ᅳ, ##ᆯ, ##ᄋ, ##ᅦ, ##ᄉ, ##ᅥ, ##ᄂ,...","[ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄌ, ##...",14,20,33
1,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_s5sSaKdCH3,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,QUES_ncJEA1ANCe,IMD평가에 포함되었던 기술관리부분의 한국의 순위는 평균 몇 위야,평균 32위,420.0,False,현위치와 관련된 평가부문은 IMD 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,현위치와 관련된 평가부문은 imd 평가에서나 종합과학기술력의 평가에서 도 포함된 부...,"[[UNK], [UNK], ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄇ, ##ᅮ...","[ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅲ, ##ᆫ, [UNK]]",7,653,659
3,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_pP8k4n8yg4,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,QUES_GnQIVxXt70,기술무역액 등과 같이 가용할 수 있는 통계 수치에 전적으로 의존하는 평가가 뭐야,종합과학기술력 평가,124.0,False,IMD의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,imd의 평가나 종합과학기술력 평가는 궁극적으로는 국가간 순위를 매기기 위한 평가이...,"[im, ##d, ##ᄋ, ##ᅴ, ᄑ, ##ᅧ, ##ᆼ, ##ᄀ, ##ᅡ, ##ᄂ...","[ᄌ, ##ᅩ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆸ, ##ᄀ, ##ᅪ, ##ᄒ, ##...",24,11,34
4,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_7b3M3mKZbb,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...",QUES_RcrMURMuhd,창의력을 저해하는 인사제도 요인은 뭐야,연공서열제,120.0,False,"기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","기업경영, 산학연 협력, 교육과 훈련, 세계화, 정부의 행정효율 그 어느 부문도 한...","[ᄀ, ##ᅵ, ##ᄋ, ##ᅥ, ##ᆸ, ##ᄀ, ##ᅧ, ##ᆼ, ##ᄋ, ##...","[ᄋ, ##ᅧ, ##ᆫ, ##ᄀ, ##ᅩ, ##ᆼ, ##ᄉ, ##ᅥ, ##ᄋ, ##...",13,173,185
5,CNTS_4740509086,국가 기술경쟁력 평가의 방법론과 응용,PARS_Ldm3nIToPd,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...",QUES_OhH70koIzZ,과학기술의 성과로 나타나는 기술집약제품이 수출에서 차지하는 비중 등을 바탕으로 하여...,산출지표,348.0,False,"IMD의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","imd의 기술경쟁력 평가가 1990년 중반 이후부터는 폭 넓게 인용되고 있지만, 그...","[im, ##d, ##ᄋ, ##ᅴ, ᄀ, ##ᅵ, ##ᄉ, ##ᅮ, ##ᆯ, ##ᄀ...","[ᄉ, ##ᅡ, ##ᆫ, ##ᄎ, ##ᅮ, ##ᆯ, ##ᄌ, ##ᅵ, ##ᄑ, ##ᅭ]",10,160,169


In [20]:
class TrainDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.texts = df['uncased_text'].values
        self.start_ids = df['start_position'].values
        self.end_ids = df['end_position'].values
        self.hash_index = df['content_id'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        returns = {
            'text' : self.texts[idx],
            'start' : self.start_ids[idx],
            'end' : self.end_ids[idx],
            'idx' : idx
        }
        return returns

In [21]:
class TestDataset(Dataset):
    def __init__(self, df):
        super().__init__()
        self.texts = df['uncased_text'].values
        self.hash_index = df['content_id'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        returns = {
            'text' : self.texts[idx],
            'idx' : idx
        }
        return returns

In [22]:
ds_pos_train = TrainDataset(df_train)
ds_pos_val = TrainDataset(df_val)
ds_pos_test = TestDataset(test)

In [24]:
MAX_LENGTH = 256
BATCH_SIZE = 32

In [25]:
dl_pos_train = DataLoader(ds_pos_train, batch_size=BATCH_SIZE, shuffle=True)

dl_pos_val = DataLoader(ds_pos_val, batch_size=BATCH_SIZE, shuffle=False)

dl_pos_test = DataLoader(ds_pos_test, batch_size=BATCH_SIZE, shuffle=False)


In [42]:

class BaseSuperModule(pl.LightningModule):
    def __init__(self, bertmodel, tokenizer, prediction_save_path):
        super().__init__()
        self.bertmodel = bertmodel
        self.tokenizer = tokenizer
        self.prediction_save_path = prediction_save_path

    def get_device(self):
        return self.bertmodel.state_dict()['bert.embeddings.word_embeddings.weight'].device

    def save_predictions(self, start_positions, end_positions):
        d = pd.DataFrame({'start_position':start_positions, 'end_position':end_positions})
        d.to_csv(self.prediction_save_path, index=False)

    def forward(self, batch):
        """
        Input:
            batch(dict), where
                batch['text'] = uncased text: str
                batch['idx'] = raw text: list(int)
                batch['start'] = start position indices : list(int) (for train & val batch only)
                batch['end'] = end position indices : list(int) (for train & val batch only)

        Output:
            For train batch, which has 'start' key and 'end' key:
                Tuple of (loss(int), start_score(torch.tensor), end_score(torch.tensor))
            For test batch, without 'start' key and 'end' key:
                Tuple of (start_score(torch.tensor), end_score(torch.tensor))
        """
        encoded_batch = tokenizer.batch_encode_plus(batch['text'], max_length=MAX_LENGTH, pad_to_max_length=True)
        input_ids = torch.tensor(encoded_batch['input_ids']).to(self.get_device())
        attention_mask = torch.tensor(encoded_batch['attention_mask']).to(self.get_device())
        start_positions = batch['start'].to(self.get_device()) + 1  if 'start' in batch.keys() else None
        end_positions = batch['end'].to(self.get_device()) + 1  if 'end' in batch.keys() else None

        model_inputs = {
            'input_ids' : input_ids,
            'attention_mask' : attention_mask,
            'start_positions' : start_positions,
            'end_positions' : end_positions
        }
        
        return self.bertmodel(**model_inputs)

    def training_step(self, batch, batch_nb):
        """
        (batch) -> (dict or OrderedDict)
        # Caution: key for loss function must exactly be 'loss'.
        """
        idx = batch['idx']
        loss = self.forward(batch)[0]
        return {'loss':loss, 'idx':idx}

    def validation_step(self, batch, batch_nb):
        """
        (batch) -> (dict or OrderedDict)
        # Caution: key for loss function must exactly be 'loss'.
        """
        idx = batch['idx']
        loss = self.forward(batch)[0]
        return {'loss':loss, 'idx':idx}

    def test_step(self, batch, batch_nb):
        """
        (batch) -> (dict or OrderedDict)
        """
        idx = batch['idx']
        start_scores = self.forward(batch)[0]
        end_scores = self.forward(batch)[1]
        return {'start_scores':start_scores, 'end_scores':end_scores, 'idx':idx}

    def training_end(self, outputs):
        """
        outputs(dict) -> loss(dict or OrderedDict)
        # Caution: key must exactly be 'loss'.
        """
        return {'loss':outputs['loss']}

    def validation_end(self, outputs):
        """
        For single dataloader:
            outputs(list of dict) -> (dict or OrderedDict)
        For multiple dataloaders:
            outputs(list of (list of dict)) -> (dict or OrderedDict)
        """        
        return {'loss':torch.mean(torch.tensor([output['loss'] for output in outputs])).detach()}

    def test_end(self, outputs):
        """
        For single dataloader:
            outputs(list of dict) -> (dict or OrderedDict)
        For multiple dataloaders:
            outputs(list of (list of dict)) -> (dict or OrderedDict)
        """
        start_scores = torch.cat([output['start_scores'] for output in outputs]).detach().cpu().numpy()
        start_positions = np.argmax(start_scores, axis=1) - 1

        end_scores = torch.cat([output['end_scores'] for output in outputs]).detach().cpu().numpy()
        end_positions = np.argmax(end_scores, axis=1) - 1
        self.save_predictions(start_positions, end_positions)
        return {}

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=2e-5)


In [43]:
class PositiveModule(BaseSuperModule):
    def __init__(self, bertmodel, tokenizer, prediction_save_path):
        super().__init__(bertmodel, tokenizer, prediction_save_path)


In [44]:
pos_module = PositiveModule(model, tokenizer, 'pos_pred.csv')

In [46]:
device = torch.device('cuda:0') if torch.cuda.is_available() else 'cpu'

In [50]:
DEBUG_MODE = False


pos_module.to(device)
pos_trainer = pl.Trainer(devices=4, accelerator="gpu")

pos_trainer.fit(pos_module, dl_pos_train, dl_pos_val )
pos_trainer.test()

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/innerwave/ailab/YYC/경진대회/lightning_logs

  | Name      | Type                     | Params
-------------------------------------------------------
0 | bertmodel | BertForQuestionAnswering | 108 M 
-------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.573   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/innerwave/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/innerwave/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `pad

Training: 0it [00:00, ?it/s]

/home/innerwave/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/home/innerwave/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1446: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(


MisconfigurationException: `.test(ckpt_path="best")` is set but `ModelCheckpoint` is not configured to save the best model.

In [ ]:
pos_pred = pd.read_csv('pos_pred.csv')

In [51]:
test

,content_id,title,paragraph_id,context,question_id,question,text,uncased_text,tokenized_text,start_position,end_position
0,CNTS_0643625675,2015년도 생활주변방사선 안전관리 실태 조사 및 분석 결과보고서,PARS_DIL8VCbM70,생활방사선법 제23조(생활주변방사선 안전관리 실태조사 및 분석) 제1항에 따른 안전...,QUES_cyOI2451l1,어디서 모나자이트 취급사업장을 생활주변방사선 실태조사 대상으로 뽑았어,생활방사선법 제23조(생활주변방사선 안전관리 실태조사 및 분석) 제1항에 따른 안전...,생활방사선법 제23조(생활주변방사선 안전관리 실태조사 및 분석) 제1항에 따른 안전...,"[ᄉ, ##ᅢ, ##ᆼ, ##ᄒ, ##ᅪ, ##ᆯ, ##ᄇ, ##ᅡ, ##ᆼ, ##...",-1,-1
1,CNTS_8295988335,청소년쉼터 이용만족도 조사,PARS_uOkQvxJVNK,또한 최근 위기청소년의 증가와 더불어 위기청소년 사회안전망 구축사업에서 가출청소년 ...,QUES_pz2vbWpWWo,어떤 문제가 위기청소년 사회안전망 형성사업에서 중요한 부분인가,또한 최근 위기청소년의 증가와 더불어 위기청소년 사회안전망 구축사업에서 가출청소년 ...,또한 최근 위기청소년의 증가와 더불어 위기청소년 사회안전망 구축사업에서 가출청소년 ...,"[[UNK], [UNK], [UNK], ᄌ, ##ᅳ, ##ᆼ, ##ᄀ, ##ᅡ, #...",-1,-1
2,CNTS_2702937576,일반계 고등학생의 진로교육활동 참여 및 만족도가 진로성숙도에 미치는 영향 소득계층...,PARS_oSXByOF16J,"고등학교 시기는 삶의 경험 속에서 지속적인 자아 탐색, 직업정보 검색 및 평가 등의...",QUES_1g3jI4y7eo,어떤 시기 때문에 고등학생 시기가 혼란스럽고 정서적 불안을 느낄 가능성이 커,"고등학교 시기는 삶의 경험 속에서 지속적인 자아 탐색, 직업정보 검색 및 평가 등의...","고등학교 시기는 삶의 경험 속에서 지속적인 자아 탐색, 직업정보 검색 및 평가 등의...","[ᄀ, ##ᅩ, ##ᄃ, ##ᅳ, ##ᆼ, ##ᄒ, ##ᅡ, ##ᆨ, ##ᄀ, ##...",-1,-1
3,CNTS_6704151053,제4차 산업혁명 시대의 물류/배송로봇의 동향 및 시사점 Logistics and D...,PARS_Fbnm80W7Ud,흥미롭게도 물류로봇과 배송로봇은 모두 세계최대의 전자상거래 기업인 Amazon에 의...,QUES_qzwOZwaeeY,Amazon은 2013년 무슨 드론을 이용한 배송 서비스를 발표했지,흥미롭게도 물류로봇과 배송로봇은 모두 세계최대의 전자상거래 기업인 Amazon에 의...,흥미롭게도 물류로봇과 배송로봇은 모두 세계최대의 전자상거래 기업인 amazon에 의...,"[ᄒ, ##ᅳ, ##ᆼ, ##ᄆ, ##ᅵ, ##ᄅ, ##ᅩ, ##ᆸ, ##ᄀ, ##...",-1,-1
4,CNTS_6718941213,장애인활동지원제도 개선방안에 관한 연구,PARS_PFujG45XmS,미국의 사례를 통해 살펴보면 현행 우리나라의 활동지원서비스는 장애인케어서비스와 유사...,QUES_hfdtXCtdzf,한국의 활동지원서비스는 미국의 예를 통해 살펴보면 뭐와 비슷할까,미국의 사례를 통해 살펴보면 현행 우리나라의 활동지원서비스는 장애인케어서비스와 유사...,미국의 사례를 통해 살펴보면 현행 우리나라의 활동지원서비스는 장애인케어서비스와 유사...,"[ᄆ, ##ᅵ, ##ᄀ, ##ᅮ, ##ᆨ, ##ᄋ, ##ᅴ, [UNK], ᄐ, ##...",-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
1621,CNTS_9511164459,2016년 고령층55~79세 노동시장 특징,PARS_tJh2Jwdosb,고령층의 자영업 비중이 감소하면서 고령층 임금근로자 비중은 증가하고 있다. 2008...,QUES_JtsKBSQITG,2016년 약 27프로로 꾸준히 증가하는 추세를 보인건 고령층의 무엇이지,고령층의 자영업 비중이 감소하면서 고령층 임금근로자 비중은 증가하고 있다. 2008...,고령층의 자영업 비중이 감소하면서 고령층 임금근로자 비중은 증가하고 있다. 2008...,"[ᄀ, ##ᅩ, ##ᄅ, ##ᅧ, ##ᆼ, ##ᄎ, ##ᅳ, ##ᆼ, ##ᄋ, ##...",-1,-1
1622,CNTS_9511164459,2016년 고령층55~79세 노동시장 특징,PARS_5Kd58IEeoF,통계청의 가계동향조사에 의하면 60세 이상 노인 단독 가구의 처분가능소득(2015년...,QUES_IajaDLmxvq,어떤 소득이 60세 이상 독거 노인 가구의 가처분소득에서 가장 높은 비율을 차지해,통계청의 가계동향조사에 의하면 60세 이상 노인 단독 가구의 처분가능소득(2015년...,통계청의 가계동향조사에 의하면 60세 이상 노인 단독 가구의 처분가능소득(2015년...,"[[UNK], [UNK], ᄋ, ##ᅴ, ##ᄒ, ##ᅡ, ##ᄆ, ##ᅧ, ##ᆫ...",-1,-1
1623,CNTS_2866845900,여성의 시간배분 결정이 보육 및 교육 정책에 주는 시사점,PARS_hx3FcnKbHp,여성의 시간배분 결정에 있어서 중요한 가격요인 중 하나가 대체보육서비스 비용이다. ...,QUES_lR6hjzsptY,무엇이 여성의 시간배분 결정에 있어서 주요한 가격소이 가운데 하나일까,여성의 시간배분 결정에 있어서 중요한 가격요인 중 하나가 대체보육서비스 비용이다. ...,여성의 시간배분 결정에 있어서 중요한 가격요인 중 하나가 대체보육서비스 비용이다. ...,"[ᄋ, ##ᅧ, ##ᄉ, ##ᅥ, ##ᆼ, ##ᄋ, ##ᅴ, ᄉ, ##ᅵ, ##ᄀ,...",-1,-1
1624,CNTS_2154408662,해양수산 산업동향 2019년 12월호,PARS_DNrvnb1Mzw,노르웨이 Cermaq사는 IBM Food Trust Labeyrie사와 협력하여 연...,QUES_ACwZJGYBfp,노르웨이 Cermaq사가 어느 회사와 협력하여 연어의 생산과 유통 정보시스템을 구축했니,노르웨이 Cermaq사는 IBM Food Trust Labeyrie사와 협력하여 연...,노르웨이 cermaq사는 ibm food trust labeyrie사와 협력하여 연...,"[[UNK], ce, ##rma, ##q, ##ᄉ, ##ᅡ, ##ᄂ, ##ᅳ, ##...",-1,-1


In [ ]:
test.index = test['content_id']
test['selected_text'] = ''

test.loc[ds_pos_test.hash_index[:BATCH_SIZE if DEBUG_MODE else len(test)], 'start_position':'end_position'] = pos_pred.values
test

In [ ]:
for i in tqdm(range(len(test))):
    if test['sentiment'].iloc[i] in ('positive', 'negative'):
        tokenized_text = test['tokenized_text'].iloc[i]
        start_position = max(test['start_position'].iloc[i], 0)
        end_position = min(test['end_position'].iloc[i], len(tokenized_text) - 1)
        
        # restore original text
        selected_text = tokenizer.convert_tokens_to_string(tokenized_text[start_position : end_position + 1])
        for original_token in test['text'].iloc[i].split():
            tokenized_form = tokenizer.convert_tokens_to_string(tokenizer.tokenize(original_token))
            selected_text = selected_text.replace(tokenized_form, original_token, 1)
        
        test['selected_text'].iloc[i] = selected_text